In [72]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import shutil
import geojson
import pandas as pd
from glob import glob
from skimage.draw import polygon2mask
from tqdm import tqdm

In [73]:
json_file_list =glob('../../data/qupath/*.geojson')
image_file_path = '../../data/NIA/**/'

In [74]:
from email.mime import image


for i in tqdm(range(len(json_file_list))):
    file_name=os.path.basename(json_file_list[i]).replace('.geojson','.jpeg')
    with open(json_file_list[i]) as f:
        data = geojson.load(f)

    # GeoJSON 데이터를 pandas 데이터프레임으로 변환
    features = data['features']
    records = []
    for feature in features:
        record = feature['properties']
        record['geometry'] = feature['geometry']
        records.append(record)

    df = pd.DataFrame(records)
    image_path = glob(image_file_path+file_name)
    shutil.copy(image_path[0],'../../data/segmentation/qupath/image/'+file_name)
    img=Image.open(image_path[0])
    anno=df['geometry']
    mask=np.zeros((1024,1024,3))
    index_mask=np.zeros((1024,1024))
    for k in range(1,len(anno)):
        coordinate=np.array(anno[k]['coordinates'][0])
        coordinate[:,[0, 1]]=coordinate[:,[1, 0]]
        mask_temp=polygon2mask((1024,1024), coordinate)
        mask[...,0]+=mask_temp
        index_mask+=mask_temp*k
    mask=np.where(mask>=1,255,0)
    np_img=np.array(img)
    overlay=np_img*0.7+mask*0.3
    index_mask=index_mask.astype(np.uint32)
    np.save('../../data/segmentation/qupath/mask/'+file_name.replace('.jpeg','.npy'),index_mask)
    
    Image.fromarray(overlay.astype(np.uint8)).save('../../data/segmentation/qupath/overlay/'+file_name.replace('.jpeg','_overlay.jpeg'))

100%|██████████| 65/65 [06:02<00:00,  5.58s/it]
